# SILVA Benchmarking

Using QIIME 2 API

I will run this assuming all the base files have been imported into QIIME 2.

In [3]:
import os
import qiime2 as q2
from qiime2.plugins import rescript, feature_classifier, taxa

In [4]:
dbdir = '/home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138'

In [5]:
os.chdir(dbdir)
os.getcwd()

'/home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138'

## Set up base functions

In [6]:
def prep_silva_for_bench(taxtree,
                         taxmap,
                         taxranks,
                         seqs,
                         output_path = os.getcwd(),
                         output_basename = 'default-silva-params',
                         exclude_taxa = None,
                         num_degenerates = 5,
                         homopolymer_length = 8,
                         labels = ['Archaea', 'Bacteria', 'Eukaryota'],
                         min_lens = [900, 1200, 1400],
                         mode = 'uniq',
                         perc_identity = 1,
                         threads = 4,
                         rank_handles = 'silva'):
    
    ############ Set up
    print("Writing all output too: %s" % output_path)
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    
    taxtree = q2.Artifact.load(taxtree)
    taxmap = q2.Artifact.load(taxmap)
    taxranks = q2.Artifact.load(taxranks)
    seqs = q2.Artifact.load(seqs)
    base_ouput_path = os.path.join(output_path, output_basename)
    
    ############ parse taxonomy
    
    # with species labels
    print("Processing Taxonomy with species labels...")
    parsed_taxonomy, = rescript.actions.parse_silva_taxonomy(
                                    taxtree,
                                    taxmap,
                                    taxranks,
                                    include_species_labels=True)
    parsed_taxonomy.save(base_ouput_path + '-parsed-taxonomy.qza')
    
    # without species labels
    print("Processing Taxonomy without species labels...")
    parsed_taxonomy_nosp, = rescript.actions.parse_silva_taxonomy(
                                    taxtree,
                                    taxmap,
                                    taxranks,
                                    include_species_labels=False)
    parsed_taxonomy_nosp.save(base_ouput_path + '-nosp-parsed-taxonomy.qza')
    
    ############# check to reverse transcribe
    
    # even though cull-seqs will do this... for some cases we may opt to skip
    # cull-seqs, so, we'll just ensure we reverse transcribe here.
    if str(seqs.type) == 'FeatureData[RNASequence]': 
        print("Input sequence type is \'%s\', reverse transcibing sequence..." % str(seqs.type))
        seqs, = rescript.actions.reverse_transcribe(seqs)
        seqs.save(base_ouput_path + '-reverse-transcribed-seqs.qza')
    
    ############# remove taxonomic groups, i.e. Eukaryota
    
    # We'll used parsed_taxonomy as this has all the rank information including species.
    # But, we'll only care about Phylum-level filtering here anyway.
    if exclude_taxa:
        print("Filtering \'{0}\' from reference sequences...".format(exclude_taxa))
        seqs, = taxa.actions.filter_seqs(sequences = seqs,
                                        taxonomy = parsed_taxonomy,
                                        exclude = exclude_taxa,
                                        mode='contains')
        seqs.save(base_ouput_path + '-excl-seqs.qza')
    
    ############# cull seqs
    
    # enter `None` for any of these prams to skip step
    if all([num_degenerates, homopolymer_length]):
        print("Culling sequences...")
        seqs, = rescript.actions.cull_seqs(sequences = seqs,
                                num_degenerates = num_degenerates,
                                homopolymer_length = homopolymer_length)
        seqs.save(base_ouput_path + '-culled-seqs.qza')
    
    ############# filt seq-len by taxonomy 
    # For our purposes we'll only ever filter based species. So we can use 
    # either taxonomy file. We'll use the species-labels file.
    
    # enter `None` for any of these prams to skip this step
    if all([labels, min_lens]):
        print("Removing short sequences conditioned on taxonomy...")
        seqs, discard = rescript.actions.filter_seqs_length_by_taxon(
                                                sequences = seqs,
                                                taxonomy = parsed_taxonomy,
                                                labels = labels,
                                                min_lens = min_lens)
        seqs.save(base_ouput_path + '-filt-seqs.qza')
        discard.save(base_ouput_path + '-discard-seqs.qza')
    
    ############# Derep (w/ & w/o species labels)
    
    # enter `None` for any of these prams to skip this step
    # with species labels
    print("Dereplicating sequences with species labels...")
    derep_seqs, derep_taxa = rescript.actions.dereplicate(
                                    sequences = seqs,
                                    taxa = parsed_taxonomy,
                                    mode = mode,
                                    perc_identity = perc_identity,
                                    threads = threads,
                                    rank_handles = rank_handles)
    derep_seqs.save(base_ouput_path + '-derep-seqs.qza')
    derep_taxa.save(base_ouput_path + '-derep-taxa.qza')
     
    # without species labels
    print("Dereplicating sequences without species labels...")
    derep_seqs_nosp, derep_taxa_nosp = rescript.actions.dereplicate(
                                            sequences = seqs,
                                            taxa = parsed_taxonomy_nosp,
                                            mode = mode,
                                            perc_identity = perc_identity,
                                            threads = threads,
                                            rank_handles = rank_handles)
    derep_seqs_nosp.save(base_ouput_path + '-nosp-derep-seqs.qza')
    derep_taxa_nosp.save(base_ouput_path + '-nosp-derep-taxa.qza')
    
    print("Done.")

In [7]:
def prep_silva_amplicons_for_bench(seqs_fp,
                                   taxonomy_fp,
                                   primers = ['GTGYCAGCMGCCGCGGTAA', 'GGACTACNVGGGTWTCTAAT'],
                                   n_jobs = 4,
                                   read_orientation = 'forward',
                                   mode = 'uniq',
                                   perc_identity = 1,
                                   rank_handles = 'silva',
                                   output_path = os.getcwd(),
                                   output_suffix = 'uniq-515-806.qza'):

    ###### setup
    if not os.path.exists(output_path):
        os.mkdir(output_path)
   
    seqs = q2.Artifact.load(seqs_fp)
    taxonomy = q2.Artifact.load(taxonomy_fp)
    
    ###### reverse transcribe if needed
    if str(seqs.type) == 'FeatureData[RNASequence]': 
        print("Input sequence type is \'%s\', reverse transcibing seqeunce..." % str(seqs.type))
        seqs, = rescript.actions.reverse_transcribe(seqs)
        seqs.save(base_ouput_path + '-reverse-transcribed-seqs.qza')
    
    ###### extract amp region
    print("Extracting amplicon region...")
    extr_reads, = feature_classifier.actions.extract_reads(seqs,
                                                    f_primer = primers[0],
                                                    r_primer = primers[1],
                                                    n_jobs = n_jobs,
                                                    read_orientation = read_orientation)
    sn = os.path.splitext(seqs_fp)[0]
    seq_base_out = '-'.join([sn, 'extracted', output_suffix])
    extr_reads.save(os.path.join(output_path, seq_base_out))
    
    ###### derep seqs / tax
    print("Dereplicating sequences...")
    derep_seqs, derep_taxa = rescript.actions.dereplicate(extr_reads,
                                                          taxonomy,
                                                          mode = mode,
                                                          perc_identity = perc_identity,
                                                          threads = n_jobs,
                                                          rank_handles = rank_handles)
    
    print("Saving files...")
    # seqs
    sn = os.path.splitext(seqs_fp)[0]
    seq_base_out = '-'.join([sn, output_suffix])
    seq_out_fp = os.path.join(output_path, seq_base_out)
    derep_seqs.save(seq_out_fp)
    # taxonomy
    tn = os.path.splitext(taxonomy_fp)[0] 
    tax_base_out = '-'.join([tn, output_suffix])
    tax_out_fp = os.path.join(output_path, tax_base_out)
    derep_taxa.save(tax_out_fp)
    
    print("Done.")

# Default pipeline - without Eukaryota

## NR99 default

In [29]:
path='/home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138/silva-138-nr99-default-noeuks/'
basename='silva-nr99-default-noeuks'

In [30]:
prep_silva_for_bench('silva-138-ssu-taxtree.qza',
                     'silva-138-ssu-nr99-taxmap.qza',
                     'silva-138-ssu-taxranks.qza',
                     'silva-138-ssu-nr99-seqs.qza',
                      exclude_taxa = 'Eukaryota',
                      labels = ['Archaea', 'Bacteria'],
                      min_lens = [900, 1200],
                      output_path = path,
                      output_basename = basename)

Writing all output too: /home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138/silva-138-nr99-default-noeuks/
Processing Taxonomy with species labels...


/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Processing Taxonomy without species labels...
Input sequence type is 'FeatureData[RNASequence]', reverse transcibing sequence...
Filtering 'Eukaryota' from reference sequences...
Culling sequences...
Removing short sequences conditioned on taxonomy...
Dereplicating sequences with species labels...
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --derep_fulllength /tmp/qiime2-archive-do6otl6t/0ee4107d-bb63-49c3-a885-a23b5b7b2842/data/dna-sequences.fasta --output /tmp/tmpz6ef0ww5 --uc /tmp/tmppz41ozt5 --qmask none --xsize --threads 4



/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['Taxon'] = uc['seqID'].apply(lambda x: taxa.loc[x])
/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['centroidtaxa'] = uc['centroidID'].apply(lambda x: taxa.loc[x])


Dereplicating sequences without species labels...
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --derep_fulllength /tmp/qiime2-archive-do6otl6t/0ee4107d-bb63-49c3-a885-a23b5b7b2842/data/dna-sequences.fasta --output /tmp/tmpe4pv5tfv --uc /tmp/tmpby4dmzqh --qmask none --xsize --threads 4



/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['Taxon'] = uc['seqID'].apply(lambda x: taxa.loc[x])
/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['centroidtaxa'] = uc['centroidID'].apply(lambda x: taxa.loc[x])


Done.


# NR99 default - poor species labels (`-sl-`) removed 
Remove sequences with poor species labels. Such as:
```
's__unidentified', 'hloroplast', 'itochondria', 's__uncultured', 's__uncultivated',
'etagenome', 'andidatus', 'ryza_sativa', '_bacterium', '_proteobacterium', 'manure',
'arctic', 'marine', 'water', 'gut', 'symbiont', 'oral', 'lake', 'sea',
'microbial_mat', 'glacial', 'drainage', 'thermal_vent', 'nrichment', 
'synthetic', 'candidate', 'clone', 'mineralizing', 'swine', 'isolate',
'aerobic', 'hot_spring', 'halophilic', 'gas_vacuolate'
```

Note: we are not making use of Eukaryotes in this notebook, so removing 'ryza_sativa', etc.. will be fine.

In [8]:
path='/home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138/silva-138-nr99-default-noeuks-sl/'
basename='silva-nr99-default-noeuks-sl'

In [9]:
prep_silva_for_bench('silva-138-ssu-taxtree.qza',
                     'silva-138-ssu-nr99-taxmap.qza',
                     'silva-138-ssu-taxranks.qza',
                     'silva-138-ssu-nr99-seqs.qza',
                      exclude_taxa = 'Eukaryota,s__unidentified,hloroplast,itochondria,s__uncultured,s__uncultivated,etagenome,andidatus,ryza_sativa,_bacterium,_proteobacterium,manure,arctic,marine,water,gut,symbiont,oral,lake,sea,microbial_mat,glacial,drainage,thermal_vent,nrichment,synthetic,candidate,clone,mineralizing,swine,isolate,aerobic,hot_spring,halophilic,gas_vacuolate',
                      labels = ['Archaea', 'Bacteria'],
                      min_lens = [900, 1200],
                      mode = 'uniq',
                      output_path = path,
                      output_basename = basename)

Writing all output too: /home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138/silva-138-nr99-default-noeuks-sl/
Processing Taxonomy with species labels...


/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Processing Taxonomy without species labels...
Input sequence type is 'FeatureData[RNASequence]', reverse transcibing sequence...
Filtering 'Eukaryota,s__unidentified,hloroplast,itochondria,s__uncultured,s__uncultivated,etagenome,andidatus,ryza_sativa,_bacterium,_proteobacterium,manure,arctic,marine,water,gut,symbiont,oral,lake,sea,microbial_mat,glacial,drainage,thermal_vent,nrichment,synthetic,candidate,clone,mineralizing,swine,isolate,aerobic,hot_spring,halophilic,gas_vacuolate' from reference sequences...
Culling sequences...
Removing short sequences conditioned on taxonomy...
Dereplicating sequences with species labels...
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --derep_fulllength /home/mrobeson/tmp/qiime2-archive-ylcvs1cz/e7da981c-cfc2-4d7b-8016-b101cc0c77ec/data/dna-sequences.fasta --o

/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['Taxon'] = uc['seqID'].apply(lambda x: taxa.loc[x])
/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['centroidtaxa'] = uc['centroidID'].apply(lambda x: taxa.loc[x])


Dereplicating sequences without species labels...
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --derep_fulllength /home/mrobeson/tmp/qiime2-archive-ylcvs1cz/e7da981c-cfc2-4d7b-8016-b101cc0c77ec/data/dna-sequences.fasta --output /home/mrobeson/tmp/tmpo20bbgw1 --uc /home/mrobeson/tmp/tmpbgi25a_0 --qmask none --xsize --threads 4



/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['Taxon'] = uc['seqID'].apply(lambda x: taxa.loc[x])
/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['centroidtaxa'] = uc['centroidID'].apply(lambda x: taxa.loc[x])


Done.


## Ful db - default - no euks

**skip w/ euks for now**

In [11]:
path='/home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138/silva-138-raw-default-noeuks/'
basename='silva-raw-default-noeuks'

In [12]:
prep_silva_for_bench('silva-138-ssu-taxtree.qza',
                     'silva-138-ssu-taxmap.qza',
                     'silva-138-ssu-taxranks.qza',
                     'silva-138-ssu-seqs.qza',
                      exclude_taxa = 'Eukaryota',
                      labels = ['Archaea', 'Bacteria'],
                      min_lens = [900, 1200],
                      output_path = path,
                      output_basename = basename,)

Writing all output too: /home/mrobeson/projects/rescript_benchmarks/ref_dbs/silva-138/silva-138-raw-default-noeuks/
Processing Taxonomy with species labels...
Processing Taxonomy without species labels...
Input sequence type is 'FeatureData[RNASequence]', reverse transcibing sequence...
Filtering 'Eukaryota' from reference sequences...
Culling sequences...
Removing short sequences conditioned on taxonomy...
Dereplicating sequences with species labels...
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --derep_fulllength /home/mrobeson/tmp/qiime2-archive-wcjv9q8x/42e46003-c5ec-4813-aedd-fc86a00e9661/data/dna-sequences.fasta --output /home/mrobeson/tmp/tmpm4atpvcg --uc /home/mrobeson/tmp/tmpfwkdn3d2 --qmask none --xsize --threads 4



/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['Taxon'] = uc['seqID'].apply(lambda x: taxa.loc[x])
/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['centroidtaxa'] = uc['centroidID'].apply(lambda x: taxa.loc[x])


Dereplicating sequences without species labels...
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --derep_fulllength /home/mrobeson/tmp/qiime2-archive-wcjv9q8x/42e46003-c5ec-4813-aedd-fc86a00e9661/data/dna-sequences.fasta --output /home/mrobeson/tmp/tmpu2msf3bw --uc /home/mrobeson/tmp/tmpb6e7lsi8 --qmask none --xsize --threads 4



/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['Taxon'] = uc['seqID'].apply(lambda x: taxa.loc[x])
/home/mrobeson/software/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/rescript/dereplicate.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc['centroidtaxa'] = uc['centroidID'].apply(lambda x: taxa.loc[x])


Done.
